# BNF NOTATION (PART 1)


\begin{eqnarray}
    \textit{mod}     & ::= & \textbf{ Module( } \textit{ statement } \textbf{ ) }\\
                         &     & \\
    \textit{ statement } & ::= & \textbf{ FunctionDef( } \textit{ statement } \textbf{,} \textit{ argument }\textbf{ ) };\\
                         & |   & \textbf{ Return( } \textit{ expression } \textbf{ )}\\
                         & |   & \textbf{ Assign( } \textit{ expression } \textbf{,} \textit{ expression} \textbf{ )}\\
                         & |   & \textbf{ Import( } \textit{ strings } \textbf{ )}\\
                         & |   & \textbf{ Expr( } \textit{ expr } \textbf{ )}\\
                         &     & \\
    \textit{expr}        & ::= & \textbf{Add( } \textit{ expr, operator, expr } \textbf{ )} \\
                         & |   & \textbf{Num}\\
                         & |   & \textbf{Name}\\
                         & |   & \textbf{String}\\
                         & |   & \textbf{Attribute ( } \textit{ name } \textbf{ )}\\
                         & |   & \textbf{List ( } \textit{ element } \textbf{ )}\\
                         & |   & \textbf{Call(} \textit{ Attribute } \textbf{ )}\\
                         & |   & \textbf{Tuple( } \textit{ expr, expr } \textbf{ )}\\
                         & |   & \textbf{List( } \textit{ exprs } \textbf{ )}\\
                         & |   & \\
    \textit{operator}    & ::= & \textbf{Add}\\
                         & |   & \textbf{Sub}\\
                         & |   & \textbf{Mult}\\
                         & |   & \textbf{Div}\\
    \textit{argument}    & ::= & \textbf{args}\\
    \textit{arg}    & ::= & \textit{ (expr, expr) } \\
\end{eqnarray}

# CHECK IF VALID IN OUR LANG (PART 2)

Below is our validity checker for our language. As seen in the BNF, our language allows only numbers, lists, and variables, with basic operations and assignments. This BNF allows us to use most of scipy (excluding boolops and a few other aspects, basically it allows us to do methods dependent on str, var, and num)

In [164]:
import ast
import sympy as sp
import inspect

#keywords for our environment that are imported
#other things that will be added to environment include variables
kwords = {'sp','inspect','ast', 'cos', 'sin'}
# this is our parser for the subset chosen, really we only care about
# numbers, basic strings, variables, and common AST needed when 
# doing summations/product 
class valid_lang(ast.NodeVisitor):
    
    def __init__(self,kwords={}, write=False):
        #constructor, keep track of keywords and environment variables
        self.keywords = {'print', 'str','cos'}.union(kwords)
        self.env = {}
        self.write = write
       
    
    def add_env(self,variable,value):
        #checks if variable exists in environment, if no, add, if yes, do nothing
        if variable not in self.env:
            self.env[variable] = value
        else:
            pass
        
    def visit_Module(self, node):
        #creates list of ast to parse and evaluates all
        
        if self.write:
            print(type(node)) 
        all_visits = [self.visit(expr) for expr in node.body]
        if self.write:
            print()
            print('Results: ' + str(all_visits))
            print('Environment: ' + str(self.env))
        return all(all_visits)
    
    
    def visit_FunctionDef(self, node):
    
        # adds variable to env
        if self.write:
            print(str(type(node)) + ' : ' + node.name + str([arg.arg for arg in node.args.args])) 
        self.add_env(node.name, True)
        results = [self.visit(stmt) for stmt in node.body]
        
        if self.write:
            print(results)
        return all(results)
        
    
    def visit_Assign(self, node):
        #checks for assign statements
        #for simplicity, lets assume there can only be one assignment per call
        #add
        if self.write:
            print(type(node))  
        self.add_env(node.targets[0].id,True)
        #check the assigned for validity
        self.visit(node.targets[0])
        #return the validity of assigner
        return self.visit(node.value)
    
    def visit_Call(self, node):
        #we want to make sure the function called and arguments = valid
        if self.write:
            print(type(node))
        return self.visit(node.func) and all([self.visit(x) for x in node.args])
    
    def visit_Str(self, node):
        #we don't care what it is, just that it's a string, return true
        if self.write:
            print(type(node))
        return True
    
    def visit_Num(self, node):
        #we allow num, so doesn't matter as long as it is one, return true
        if self.write:
            print(type(node)) 
        return True

    def visit_Name(self, node):
        #check if the name is in keywords or in environment
        name = node.id
        if self.write:
            print(str(type(node)) + ' : ' + name)
        
        return (name in self.keywords) or (name in self.env)
        
    def visit_Tuple(self,node):
        if self.write:
            print(type(node))      
        return [self.visit(elt) for elt in node.elts]
    
    def visit_List(self,node):
        if self.write:
            print(type(node))      
        return [self.visit(elt) for elt in node.elts]
    
    def visit_Expr(self, node):
        #when we get expression, don't care what for, just that is valid in lang
        if self.write:
            print(type(node))
        return self.visit(node.value)
    
    def visit_BinOp(self, node):
        #we allow operators, so no need to check, just that each part is valid
        if self.write:
            print(type(node)) 
        if type(node.op) in {ast.Add, ast.Sub, ast.Mult, ast.Div}:
            return (self.visit(node.left)) and (self.visit(node.right))
    def visit_Import(self, node):
        #we want to add import to enviornment
        if self.write:
            print(str(type(node)) + ' : ' + str([alias.asname for alias in node.names]))
        self.add_env(node.names[0].asname,node.names[0].name)
        return True
        
    def visit_Attribute(self, node):
        #we don't care about what attribute for, just that it is valid
        if self.write:
            print(str(type(node)) + ' : ' + node.attr)
        return self.visit(node.value)
    
    
    def visit_Return(self, node):
        #another easy one, don't care about anything other than is inside expression valid
        if self.write:
            print(type(node))  
        return self.visit(node.value)

 
        
    
#okay now we have the parser, but we want to make the wrapper function
#so we can just preprend it to any function we work with to check validity

def validity(kwords={}, write=True):
    def wrapper(x):
        node = ast.parse(inspect.getsource(x))
        valid = valid_lang(kwords, write=True).visit(node)
        print("The Object " + str(x) + " has: " + ("Valid Syntax" if valid else "Invalid Syntax"))
    return wrapper


# test to see what to include in is_Valid


#source = inspect.getsource(summation_sympy)
#abstract_syntax_tree = ast.parse(source)
#print(ast.dump(abstract_syntax_tree))
#print(ast.dump(ast.parse('import scipy as sp')))
print("This shows that our parser works for all mathematical types/operands we allow")
print("Additionally, it shows we can successfully call methods, assign variables, import, and return, etc.")
@validity(kwords, write=True)
def a():
    import scipy as sp
    x = sp.arrange(3,5)
    return x

#source = inspect.getsource(a)
#print(source)
#abstract_syntax_tree = ast.parse(source)

#valid_lang().visit(abstract_syntax_tree)

#print()
#print(ast.dump(abstract_syntax_tree))

#source = inspect.getsource(a)
#abstract_syntax_tree = ast.parse(source)
#valid_lang(write=True).visit(abstract_syntax_tree)

#print()
#print(ast.dump(abstract_syntax_tree))

This shows that our parser works for all mathematical types/operands we allow
Additionally, it shows we can successfully call methods, assign variables, import, and return, etc.
<class '_ast.Module'>
<class '_ast.FunctionDef'> : a[]
<class '_ast.Import'> : ['sp']
<class '_ast.Assign'>
<class '_ast.Name'> : x
<class '_ast.Call'>
<class '_ast.Attribute'> : arrange
<class '_ast.Name'> : sp
<class '_ast.Num'>
<class '_ast.Num'>
<class '_ast.Return'>
<class '_ast.Name'> : x
[True, True, True]

Results: [True]
Environment: {'a': True, 'sp': 'scipy', 'x': True}
The Object <function a at 0x112246488> has: Valid Syntax


# Algorithms and Analysis (Part 3)

Ok, this is where it gets interesting. Taking our known validity checker, we want to alter the returns to do meaningful operations with our non-trivial algorithm that will manipulate matrices. We want to do this without running the actual function, just inspect ast.

This first change is essentially a counter for our functions number of calls to anything in the AST. This could be useful because we are now able to check if it is valid in our language, and then roughly how long it will take to run. If we know the efficiency of the functions we call and AST actions (assuming we do) we could now prevent running code that would be ineffectively slow (which could cause issues in large quantities of code where having to restart and optimize could waste a lot of time.

In [175]:
import ast
import sympy as sp
import inspect

#keywords for our environment that are imported
#other things that will be added to environment include variables
kwords = {'sp','inspect','ast', 'cos', 'sin'}
# this is our parser for the subset chosen, really we only care about
# numbers, basic strings, variables, and common AST needed when 
# doing summations/product 
class count_process(ast.NodeVisitor):
    
    def __init__(self,kwords={}, write=False):
        #constructor, keep track of keywords and environment variables
        self.keywords = {'print', 'str','cos'}.union(kwords)
        self.env = {}
        self.write = write
        self.count = 0
       
    
    def add_env(self,variable,value):
        #checks if variable exists in environment, if no, add, if yes, do nothing
        if variable not in self.env:
            self.env[variable] = value
        else:
            pass
        
    def visit_Module(self, node):
        #creates list of ast to parse and evaluates all
        
        if self.write:
            print(type(node)) 
        self.count += 1
        all_visits = [self.visit(expr) for expr in node.body]
        if self.write:
            print()
            print('Results: ' + str(all_visits))
            print('Environment: ' + str(self.env))
            print('Total Calls: ' + str(self.count))
        return all(all_visits)
    
    
    def visit_FunctionDef(self, node):
    
        # adds variable to env
        if self.write:
            print(str(type(node)) + ' : ' + node.name + str([arg.arg for arg in node.args.args])) 
        self.count += 1
        self.add_env(node.name, True)
        results = [self.visit(stmt) for stmt in node.body]
        
        if self.write:
            print(results)
        return all(results)
        
    
    def visit_Assign(self, node):
        #checks for assign statements
        #for simplicity, lets assume there can only be one assignment per call
        #add
        if self.write:
            print(type(node))  
        self.count += 1
        self.add_env(node.targets[0].id,True)
        #check the assigned for validity
        self.visit(node.targets[0])
        #return the validity of assigner
        return self.visit(node.value)
    
    def visit_Call(self, node):
        #we want to make sure the function called and arguments = valid
        if self.write:
            print(type(node))
        self.count += 1
        return self.visit(node.func) and all([self.visit(x) for x in node.args])
    
    def visit_Str(self, node):
        #we don't care what it is, just that it's a string, return true
        if self.write:
            print(type(node))
        self.count += 1
        return True
    
    def visit_Num(self, node):
        #we allow num, so doesn't matter as long as it is one, return true
        if self.write:
            print(type(node)) 
        self.count += 1
        return True

    def visit_Name(self, node):
        #check if the name is in keywords or in environment
        name = node.id
        if self.write:
            print(str(type(node)) + ' : ' + name)
        self.count += 1
        
        return (name in self.keywords) or (name in self.env)
        
    def visit_Tuple(self,node):
        if self.write:
            print(type(node))    
        self.count += 1
        return [self.visit(elt) for elt in node.elts]
    
    def visit_List(self,node):
        if self.write:
            print(type(node)) 
        self.count += 1
        return [self.visit(elt) for elt in node.elts]
    
    def visit_Expr(self, node):
        #when we get expression, don't care what for, just that is valid in lang
        if self.write:
            print(type(node))
        self.count += 1
        return self.visit(node.value)
    
    def visit_BinOp(self, node):
        #we allow operators, so no need to check, just that each part is valid
        if self.write:
            print(type(node)) 
        
        if type(node.op) in {ast.Add, ast.Sub, ast.Mult, ast.Div}:
            self.count += 1
            return (self.visit(node.left)) and (self.visit(node.right))
    def visit_Import(self, node):
        #we want to add import to enviornment
        if self.write:
            print(str(type(node)) + ' : ' + str([alias.asname for alias in node.names]))
        self.add_env(node.names[0].asname,node.names[0].name)
        self.count += 1
        return True
        
    def visit_Attribute(self, node):
        #we don't care about what attribute for, just that it is valid
        if self.write:
            print(str(type(node)) + ' : ' + node.attr)
        self.count += 1
        return self.visit(node.value)
    
    
    def visit_Return(self, node):
        #another easy one, don't care about anything other than is inside expression valid
        if self.write:
            print(type(node))  
        self.count += 1
        return self.visit(node.value)

 
        
    
#okay now we have the parser, but we want to make the wrapper function
#so we can just preprend it to any function we work with to check validity

def giveMeCount(kwords={}, write=True):
    def wrapper(x):
        node = ast.parse(inspect.getsource(x))
        valid = count_process(kwords, write=True).visit(node)
        
    return wrapper



@giveMeCount(kwords, write=True)
def a():
    import scipy as sp
    x = sp.arrange(3,5)
    return x



<class '_ast.Module'>
<class '_ast.FunctionDef'> : a[]
<class '_ast.Import'> : ['sp']
<class '_ast.Assign'>
<class '_ast.Name'> : x
<class '_ast.Call'>
<class '_ast.Attribute'> : arrange
<class '_ast.Name'> : sp
<class '_ast.Num'>
<class '_ast.Num'>
<class '_ast.Return'>
<class '_ast.Name'> : x
[True, True, True]

Results: [True]
Environment: {'a': True, 'sp': 'scipy', 'x': True}
Total Calls: 12


For this second change, we are seeing if our function holds one uniform type (and is a type we allow). For my language this is simply strings and numbers. To simplify I marked float and int as the same type (marked as Int) because we don't care if it is all floats or ints, just that it is all num. This is useful for long functions in our language, as it allows us to infer type without needing to run through it mentally or struggle with type checking

In [191]:
import ast
import sympy as sp
import inspect

#keywords for our environment that are imported
#other things that will be added to environment include variables
kwords = {'sp','inspect','ast', 'cos', 'sin'}
# this is our parser for the subset chosen, really we only care about
# numbers, basic strings, variables, and common AST needed when 
# doing summations/product 
class count_and_type(ast.NodeVisitor):
    
    def __init__(self,kwords={}, write=False):
        #constructor, keep track of keywords and environment variables
        self.keywords = {'print', 'str','cos'}.union(kwords)
        self.env = {}
        self.write = write
        self.count = 0
        self.types = []
        self.type = None
       
    
    def add_env(self,variable,value):
        #checks if variable exists in environment, if no, add, if yes, do nothing
        if variable not in self.env:
            self.env[variable] = value
        else:
            pass
        
    def visit_Module(self, node):
        #creates list of ast to parse and evaluates all
        
        if self.write:
            print(type(node)) 
        self.count += 1
        all_visits = [self.visit(expr) for expr in node.body]
        if not any(not isinstance(y,(int,float)) for y in self.types):
            self.type = int
        if not any(not isinstance(y,(str)) for y in self.types):
            self.type = str
        if self.write:
            print()
            print('Results: ' + str(all_visits))
            print('Environment: ' + str(self.env))
            print('Total Calls: ' + str(self.count))
            print('Type List:' + str(self.types))
            print('Total Type:' + str(self.type))
        return all(all_visits)
    
    
    def visit_FunctionDef(self, node):
    
        # adds variable to env
        if self.write:
            print(str(type(node)) + ' : ' + node.name + str([arg.arg for arg in node.args.args])) 
        self.count += 1
        self.add_env(node.name, True)
        results = [self.visit(stmt) for stmt in node.body]
        
        if self.write:
            print(results)
        return all(results)
        
    
    def visit_Assign(self, node):
        #checks for assign statements
        #for simplicity, lets assume there can only be one assignment per call
        #add
        if self.write:
            print(type(node))  
        self.count += 1
        self.add_env(node.targets[0].id,True)
        #check the assigned for validity
        self.visit(node.targets[0])
        #return the validity of assigner
        return self.visit(node.value)
    
    def visit_Call(self, node):
        #we want to make sure the function called and arguments = valid
        if self.write:
            print(type(node))
        self.count += 1
        return self.visit(node.func) and all([self.visit(x) for x in node.args])
    
    def visit_Str(self, node):
        #we don't care what it is, just that it's a string, return true
        if self.write:
            print(type(node))
        self.count += 1
        self.types.append("string")
        return True
    
    def visit_Num(self, node):
        #we allow num, so doesn't matter as long as it is one, return true
        if self.write:
            print(type(node)) 
        self.count += 1
        self.types.append(1)
        return True

    def visit_Name(self, node):
        #check if the name is in keywords or in environment
        name = node.id
        if self.write:
            print(str(type(node)) + ' : ' + name)
        self.count += 1
        
        return (name in self.keywords) or (name in self.env)
        
    def visit_Tuple(self,node):
        if self.write:
            print(type(node))    
        self.count += 1
        return [self.visit(elt) for elt in node.elts]
    
    def visit_List(self,node):
        if self.write:
            print(type(node)) 
        self.count += 1
        return [self.visit(elt) for elt in node.elts]
    
    def visit_Expr(self, node):
        #when we get expression, don't care what for, just that is valid in lang
        if self.write:
            print(type(node))
        self.count += 1
        return self.visit(node.value)
    
    def visit_BinOp(self, node):
        #we allow operators, so no need to check, just that each part is valid
        if self.write:
            print(type(node)) 
        
        if type(node.op) in {ast.Add, ast.Sub, ast.Mult, ast.Div}:
            self.count += 1
            return (self.visit(node.left)) and (self.visit(node.right))
    def visit_Import(self, node):
        #we want to add import to enviornment
        if self.write:
            print(str(type(node)) + ' : ' + str([alias.asname for alias in node.names]))
        self.add_env(node.names[0].asname,node.names[0].name)
        self.count += 1
        return True
        
    def visit_Attribute(self, node):
        #we don't care about what attribute for, just that it is valid
        if self.write:
            print(str(type(node)) + ' : ' + node.attr)
        self.count += 1
        return self.visit(node.value)
    
    
    def visit_Return(self, node):
        #another easy one, don't care about anything other than is inside expression valid
        if self.write:
            print(type(node))  
        self.count += 1
        return self.visit(node.value)

 
        
    
#okay now we have the parser, but we want to make the wrapper function
#so we can just preprend it to any function we work with to check validity

def giveMeCountAndType(kwords={}, write=True):
    def wrapper(x):
        node = ast.parse(inspect.getsource(x))
        valid = count_and_type(kwords, write=True).visit(node)
        
    return wrapper



@giveMeCountAndType(kwords, write=True)
def a():
    import scipy as sp
    x = sp.arrange(3,5)
    return x

print()

@giveMeCountAndType(kwords, write=True)
def a():
    import scipy as sp
    x = "dogg"
    return x

print()

@giveMeCountAndType(kwords, write=True)
def a():
    import scipy as sp
    x = "dogg"
    y = sp.arrange(3,5)
    return (x,y)

<class '_ast.Module'>
<class '_ast.FunctionDef'> : a[]
<class '_ast.Import'> : ['sp']
<class '_ast.Assign'>
<class '_ast.Name'> : x
<class '_ast.Call'>
<class '_ast.Attribute'> : arrange
<class '_ast.Name'> : sp
<class '_ast.Num'>
<class '_ast.Num'>
<class '_ast.Return'>
<class '_ast.Name'> : x
[True, True, True]

Results: [True]
Environment: {'a': True, 'sp': 'scipy', 'x': True}
Total Calls: 12
Type List:[1, 1]
Total Type:<class 'int'>

<class '_ast.Module'>
<class '_ast.FunctionDef'> : a[]
<class '_ast.Import'> : ['sp']
<class '_ast.Assign'>
<class '_ast.Name'> : x
<class '_ast.Str'>
<class '_ast.Return'>
<class '_ast.Name'> : x
[True, True, True]

Results: [True]
Environment: {'a': True, 'sp': 'scipy', 'x': True}
Total Calls: 8
Type List:['string']
Total Type:<class 'str'>

<class '_ast.Module'>
<class '_ast.FunctionDef'> : a[]
<class '_ast.Import'> : ['sp']
<class '_ast.Assign'>
<class '_ast.Name'> : x
<class '_ast.Str'>
<class '_ast.Assign'>
<class '_ast.Name'> : y
<class '_ast.Ca